<a href="https://colab.research.google.com/github/louistrue/learn-ifc/blob/main/BFH-25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Einrichtung und Installation
Bevor wir mit der Bearbeitung von IFC-Modellen beginnen, muessen wir sicherstellen, dass `ifcopenshell` und `lark-parser` installiert sind. Diese Bibliotheken ermoeglichen es uns, IFC-Dateien effektiv zu bearbeiten und zu analysieren.


In [15]:
!pip install ifcopenshell
!pip install lark-parser

### Erstellen eines neuen IFC-Modells
Hier initialisieren wir ein neues IFC-Modell und erstellen ein Grundelement, das `IfcProject`, welches fuer die Organisation und Speicherung unserer Bauinformationen notwendig ist.

Nach dem Aufbau der Grundstruktur des Modells konstruieren wir eine Wand mit spezifischen Massen und ordnen diese der Etage zu. Ausserdem wird das Material fuer die Wand festgelegt.

Nachdem das IFC-Modell erfolgreich erstellt und gespeichert wurde, stellen wir es zur Verfuegung, damit es heruntergeladen wird.

In [16]:
import ifcopenshell
from ifcopenshell.api import run
import datetime

# Ein leeres IFC-Modell erstellen
model = ifcopenshell.file()

# Projekt erstellen
project = run("root.create_entity", model, ifc_class="IfcProject", name="Mein Projekt")

# Metrische Einheiten zuweisen
run("unit.assign_unit", model)

# Geometrischer Kontext (3D)
context = run("context.add_context", model, context_type="Model")
body = run("context.add_context", model, context_type="Model", context_identifier="Body", target_view="MODEL_VIEW", parent=context)

# Hierarchie Standort - Gebaeude - Etage
site = run("root.create_entity", model, ifc_class="IfcSite", name="Projektstandort")
building = run("root.create_entity", model, ifc_class="IfcBuilding", name="Gebaeude A")
storey = run("root.create_entity", model, ifc_class="IfcBuildingStorey", name="Erdgeschoss")
run("aggregate.assign_object", model, relating_object=project, products=[site])
run("aggregate.assign_object", model, relating_object=site, products=[building])
run("aggregate.assign_object", model, relating_object=building, products=[storey])

# Wand erzeugen
wall = run("root.create_entity", model, ifc_class="IfcWall", name="Meine coole Wand")
run("geometry.edit_object_placement", model, product=wall)
representation = run("geometry.add_wall_representation", model, context=body, length=5, height=3, thickness=0.2)
run("geometry.assign_representation", model, product=wall, representation=representation)

# Material der Wand zuweisen
material = model.createIfcMaterial("Beton")
material_layer = model.createIfcMaterialLayer(material, 200, None)  # Dicke in mm (nur symbolisch)
material_layer_set = model.createIfcMaterialLayerSet([material_layer], None)
material_layer_set_usage = model.createIfcMaterialLayerSetUsage(material_layer_set, "AXIS2", "POSITIVE", -100)
model.createIfcRelAssociatesMaterial(ifcopenshell.guid.new(), None, "Material zu Wand", None, [wall], material_layer_set_usage)

# Wand der Etage zuordnen
model.createIfcRelContainedInSpatialStructure(
    GlobalId=ifcopenshell.guid.new(),
    RelatedElements=[wall],
    RelatingStructure=storey
)

# Speichern
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"/content/einfache_Wand_{current_time}.ifc"
model.write(filename)
print(f"Das Modell wurde erfolgreich erstellt und unter '{filename}' gespeichert.")

from google.colab import files
files.download(filename)

TypeError: Incorrect function arguments provided for aggregate.assign_object
assign_object() got an unexpected keyword argument 'product'. You specified args (<ifcopenshell.file.file object at 0x7a7dc9587740>,) and settings {'relating_object': #1=IfcProject('05k4wW_WvC68qFuaNfh3Ip',$,'Mein Projekt',$,$,$,$,(#10),#5), 'product': #12=IfcSite('0f_1jw6Wv7hACkwp1nfqz5',$,'Projektstandort',$,$,$,$,$,$,$,$,$,$,$)}

Correct signature is (file: ifcopenshell.file.file, products: list[ifcopenshell.entity_instance.entity_instance], relating_object: ifcopenshell.entity_instance.entity_instance) -> Optional[ifcopenshell.entity_instance.entity_instance]
See help(ifcopenshell.api.aggregate.assign_object) for documentation.

### Fensterplatzierung
Nachdem wir die Wand erstellt haben, oeffnen wir die zuvor erstellte Datei und fuegen ein Fenster hinzu. Danach speichern wir wieder als neue Datei.

In [ ]:
import ifcopenshell
from ifcopenshell.api import run

# IFC-Modell laden
model = ifcopenshell.open(filename)
storey = model.by_type("IfcBuildingStorey")[0] if model.by_type("IfcBuildingStorey") else None
if not storey:
    raise SystemExit("Keine Gebaeudeetage im Modell gefunden.")

# Wand finden
walls = model.by_type("IfcWall")
if not walls:
    raise SystemExit("Keine Wand im Modell gefunden.")
wall = walls[0]

# Fensterparameter (mm) -> wir konvertieren unten nach m
window_params = {
    "name": "Neues Fenster",
    "breite": 1800,
    "tiefe": 250,
    "hoehe": 1200,
    "offset_x": 1500,
    "offset_y": 50,
    "offset_z": 500
}

to_m = lambda v_mm: float(v_mm) / 1000.0

# Platzierung relativ zur Wand
wall_placement = wall.ObjectPlacement
wall_location = wall_placement.RelativePlacement.Location
placement_point = model.createIfcCartesianPoint((
    wall_location.Coordinates[0] + to_m(window_params["offset_x"]),
    wall_location.Coordinates[1] + to_m(window_params["offset_y"]),
    wall_location.Coordinates[2] + to_m(window_params["offset_z"]) ))
axis2placement = model.createIfcAxis2Placement3D(placement_point)
window_placement = model.createIfcLocalPlacement(wall_placement, axis2placement)

# Fenstergeometrie erstellen (rein visuell)
# Create proper 2D axis placement for the profile
profile_point = model.createIfcCartesianPoint((0.0, 0.0))
profile_axis = model.createIfcAxis2Placement2D(profile_point)
rect_profile = model.createIfcRectangleProfileDef("AREA", "BaseProfile", profile_axis, to_m(window_params["breite"]), to_m(window_params["tiefe"]))
extrude_dir = model.createIfcDirection((0.0, 0.0, 1.0))
extruded_area = model.createIfcExtrudedAreaSolid(rect_profile, profile_axis, extrude_dir, to_m(window_params["hoehe"]))
rep = model.createIfcShapeRepresentation(model.by_type("IfcGeometricRepresentationContext")[0], "Body", "SweptSolid", [extruded_area])
product_def_shape = model.createIfcProductDefinitionShape(None, None, [rep])

# Fensterentitaet
new_window = model.createIfcWindow(ifcopenshell.guid.new(), None, window_params["name"], None, None, window_placement, product_def_shape, None)
model.createIfcRelContainedInSpatialStructure(ifcopenshell.guid.new(), None, "Contains", None, [new_window], storey)

# Farbe vergeben (optional)
color = [100/255.0, 149/255.0, 237/255.0]
style = run("style.add_style", model, name="DynamicStyle")
run("style.add_surface_style", model, style=style, ifc_class="IfcSurfaceStyleShading", attributes={
    "SurfaceColour": {"Name": None, "Red": color[0], "Green": color[1], "Blue": color[2]}
})
run("style.assign_representation_styles", model, shape_representation=rep, styles=[style])

# Speichern
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"/content/Wand_mit_Fenster_{current_time}.ifc"
model.write(filename)
print("Updated model saved to:", filename)
from google.colab import files
files.download(filename)

### Fensteroeffnung in die Wand schneiden (Schritt 3)
In diesem Schritt wird eine `IfcOpeningElement` erstellt und ueber `IfcRelVoidsElement` mit der Wand verknuepft. Dadurch entsteht eine echte Oeffnung in der Wandgeometrie. Wir nutzen dieselben Parameter und Offsets wie beim Fenster.

In [ ]:
import ifcopenshell
import datetime

# Vorherige Ausgabedatei oeffnen
model = ifcopenshell.open(filename)

# Wand holen
walls = model.by_type("IfcWall")
if not walls:
    raise SystemExit("Keine Wand im Modell gefunden.")
wall = walls[0]

# Sicherstellen, dass window_params existiert (aus Schritt 2)
try:
    window_params
except NameError:
    # Fallback: Standardwerte falls das Notebook direkt in Schritt 3 gestartet wird
    window_params = {
        "breite": 1800,
        "tiefe": 250,
        "hoehe": 1200,
        "offset_x": 1500,
        "offset_y": 50,
        "offset_z": 500
    }

to_m = lambda v_mm: float(v_mm) / 1000.0

# Platzierung fuer die Oeffnung relativ zur Wand wie beim Fenster
wall_placement = wall.ObjectPlacement
wall_location = wall_placement.RelativePlacement.Location
opening_point = model.createIfcCartesianPoint((
    wall_location.Coordinates[0] + to_m(window_params["offset_x"]),
    wall_location.Coordinates[1] + to_m(window_params["offset_y"]),
    wall_location.Coordinates[2] + to_m(window_params["offset_z"]) ))
opening_axis = model.createIfcAxis2Placement3D(opening_point)

# Profil und Extrusion fuer die Oeffnung (Breite x Tiefe, Extrusion in Z = Hoehe)
# Tiefe darf die Wand staerke ueberschreiten, damit sicher voll ausgeschnitten wird
opening_depth_m = to_m(window_params["tiefe"])  # z.B. 0.25 m fuer 0.2 m Wand
rect_open = model.createIfcRectangleProfileDef("AREA", "OpeningProfile", opening_axis, to_m(window_params["breite"]), opening_depth_m)
extrude_dir = model.createIfcDirection((0.0, 0.0, 1.0))
opening_solid = model.createIfcExtrudedAreaSolid(rect_open, opening_axis, extrude_dir, to_m(window_params["hoehe"]))

# Repr. fuer die Oeffnung anlegen
geom_context = model.by_type("IfcGeometricRepresentationContext")[0]
rep_open = model.createIfcShapeRepresentation(geom_context, "Body", "SweptSolid", [opening_solid])
pds_open = model.createIfcProductDefinitionShape(None, None, [rep_open])

# IfcOpeningElement erzeugen und mit Wand verknuepfen
opening = model.createIfcOpeningElement(ifcopenshell.guid.new(), None, "Fensteroeffnung", None, None, wall_placement, pds_open, None)
model.createIfcRelVoidsElement(ifcopenshell.guid.new(), None, "Voids", None, wall, opening)

# Datei erneut speichern
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"/content/Wand_mit_Fenster_Oeffnung_{current_time}.ifc"
model.write(filename)
print("Model mit echter Oeffnung gespeichert unter:", filename)

from google.colab import files
files.download(filename)

### Bereinigung des Arbeitsbereichs
Nachdem das Modell gespeichert und heruntergeladen wurde, ist es oft sinnvoll, den Arbeitsbereich aufzuraeumen, indem man nicht mehr benoetigte Dateien entfernt. Dies hilft, die Uebersichtlichkeit zu bewahren und Speicherplatz freizugeben.

In [ ]:
import os

for file_name in os.listdir():
    if file_name.endswith('.ifc'):
        os.remove(os.path.join(os.getcwd(), file_name))
        print(f"Removed {file_name}")
